In [1]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import os.path
import csv
from sklearn.svm import SVC
from sklearn.svm import NuSVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import grid_search
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics
from sklearn import preprocessing
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from random import randint
from sklearn import tree
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import math
import time
from sklearn.naive_bayes import BernoulliNB

from metrics.Correlation import Correlation
from IO.Write import Write
from IO.Read import Read
from metrics.RSquare import RSquare
from DataSet.Dataset import DataSet


def setIdToName(aList):
    
    ids = {}
    nameToId = {}
    idToName = {}
    count = 0
    
    for i in aList:
        
        nameToId[i] = count
        idToName[count] = i
        count += 1
        
    ids['nameToId'] = nameToId
    ids['idToName'] = idToName
    
    return ids

def createNewIds(oldsnps,snps):
    
    nameToId = {}
    idToName = {}
    newIds = {}
    
    oldIds = oldsnps['snps']['idToName']
    
    for i in range(len(snps)):
        
        nameToId[oldIds[snps[i]]] = i
        idToName[i] = oldIds[snps[i]]
        
    newIds['nameToId'] = nameToId
    newIds['idToName'] = idToName
        
    return newIds

def setSnpsCode(patients,chromosomes):
    
    for i in patients.keys():
        patients[i].snpCode(chromosomes)
        
    return patients
    

def tables(sampleX,sampleY,k):
  
    samples = {}
    
    for run in range(1,k+1):
        
        d1 = {}
        

        dataTestX = sampleX[run]
        dataTestY = sampleY[run]

        n = 0

        for i in sampleX.keys():

            if i != run:

                n += len(sampleX[i])

        dataTrainX = np.zeros((n,len(sampleX[1].T)),dtype = int)
        dataTrainY = np.zeros((n,),dtype = int)

        count = 0

        for sample in sampleX.keys():

            if sample != run:

                 for i in range(len(sampleX[sample])):
                    for j in range(len(sampleX[sample].T)):
                        dataTrainX[count,j] = sampleX[sample][i,j]

                    dataTrainY[count] = sampleY[sample][i]
                    count += 1

        d1['trainX'] = dataTrainX
        d1['trainY'] = dataTrainY
        d1['testX'] = dataTestX
        d1['testY'] = dataTestY
        
        samples[run] = d1
    
    return samples
    
def kSampleData(k,X,Y):
    
    x = int (len(X) / k)
    allElements = np.zeros((len(X),),dtype = int)
    
    count1 = 1
    sampleX = {}
    sampleY = {}
   
    
    while count1 <= k:
        count2 = 1
        sampleData = []
        
        if count1 == k:
            x =  len(X) - ((k-1) * x)
        
        dataX = np.zeros((x,len(X.T)),dtype = int)
        dataY = np.zeros((x,),dtype = int)
        
        while count2 <= x:
            
            aRand = randint(0,len(X)-1)
            
            while allElements[aRand] == 1:
                
                aRand = randint(0,len(X)-1)
            
            allElements[aRand] = 1
            sampleData.append(aRand)
            count2 += 1
            
        for i in range(len(sampleData)):
            for j in range(len(X.T)):
                dataX[i,j] = X[sampleData[i],j]
            
            dataY[i] = Y[sampleData[i]]
            
        sampleX[count1] = dataX
        sampleY[count1] = dataY
        count1 +=1
        
    return tables(sampleX,sampleY,k)




def createNewTable(snps,X):
    
    newX = np.zeros((len(X),len(snps)),dtype = int)
    count=0
    
    
    
    for i in range(len(newX)):
        for j in range(len(newX.T)):
            newX[i,j] = -1
    
    for i in range(len(snps)):
        
        newX[:,i] = X[:,snps[i]]
        
        
    print("new shape = ",newX.shape)
            
    return newX 



def featuresIds(oldSnps,snps):
    
    features = {}
    
    for i in range(len(snps)):
        features[i] = snps[i]
        
    return features



def crossValidiation(X, Y, k = 1, continious = True, classifier = None,OLS = False,Logistic = False):
    
    if not classifier:
        print("wrong!!!!!!! you have to choise a classifier")
        return
    
    results = {}
    accuracy = {}
    auc = {}
    recall = {}
    precision = {}
    f1Score = {}
    
    sumResults = 0.0
    sumAccuracy = 0.0
    sumAuc = 0.0
    sumRecall = 0.0
    sumPrecision = 0.0
    sumF1Score = 0.0
    
    samples = kSampleData(k,X,Y)
    
    for run in range(1, k + 1):
        
        trainX = samples[run]['trainX']
        trainY = samples[run]['trainY']
        
        #trainX,trainY = balancedData(trainX,trainY)
        
        testX = samples[run]['testX']
        testY = samples[run]['testY']
        
        
        if OLS:
            classifier = sm.OLS(trainY,trainX)
            yPredict = classifier.fit().predict(testX)
        else:

            classifier.fit(trainX, trainY)
            yPredict = classifier.predict(testX)
        
        if continious:
            
            for i in range(len(yPredict)):
                
                if (abs(0 - yPredict[i]) - abs(1 - yPredict[i])) <= 1e-10 :
                    yPredict[i] = 0
                else:
                    yPredict[i] = 1
                    
        if Logistic:
            
            probabilities = classifier.predict_proba(testX)
            
            for i in range(len(probabilities)):
                if probabilities[i][1] >= 0.8:
                    yPredict[i] = 1
                else:
                    yPredict[i] = 0
          
        accuracy[run] = metrics.accuracy_score(testY,yPredict)#(yPredict,testY)#
        fpr, tpr, thresholds = metrics.roc_curve(testY,yPredict)
        auc[run] = metrics.auc(fpr,tpr)
        recall[run] = metrics.recall_score(testY,yPredict)
        precision[run] = metrics.precision_score(testY,yPredict)
        f1Score[run] = f1_score(testY, yPredict, average='binary')
        
    
    for i in accuracy.keys():
        sumAccuracy = sumAccuracy + accuracy[i]
        sumAuc = sumAuc + auc[i]
        sumRecall = sumRecall + recall[i]
        sumPrecision = sumPrecision + precision[i]
        sumF1Score = sumF1Score + f1Score[i]
    
    results['accuracy'] = sumAccuracy / k
    results['auc'] = sumAuc / k
    results['recall'] = sumRecall / k
    results['precision'] = sumPrecision / k
    results['f1'] = sumF1Score / k
    
    return results

def writeResultConf(path,name,results):
    
    timee = time.strftime("%d-%m-%Y")
    
    folder = path + name + 'Confu' + " ( " + timee + " ) " + '\\'
    #file = path + name + 'Confu' + '.txt'
    
    i=1
    while os.path.exists(folder):
        timee = time.strftime("%d-%m-%Y")
        folder = path + name + 'Confu' + " ( " + timee + " ) " + '_' + str(i) + '\\'
    
        i += 1
        
    os.makedirs(folder)
    
    file = folder + name + 'Confu' + '.txt' 
    
    write = open(file,'w')
    
    write.write(timee + '\n'+ '\n')
    write.write(name + '\n')
    
    for i in results:
        
        results1 = results[i]
        write.write("Category = " + i + '\n')
        
        for j in results1:
            
            write.write(j + " = " + str(results1[j]) + '\n')
            
        write.write('\n')
        write.write('\n')
        
    write.close()

    

def writeResult(path,name,results):
    
    timee = time.strftime("%d-%m-%Y")
    
    folder = path + name  + " ( " + timee + " ) " + '\\'
    #file = path + name  + '.txt'
    
    i=1
    while os.path.exists(folder):
        timee = time.strftime("%d-%m-%Y")
        folder = path + name + " ( " + timee + " ) " + '_' + str(i) + '\\'
       
        i += 1
        
    os.makedirs(folder)
    
    file = folder + name + '.txt'
    
    write = open(file,'w')
    
    write.write(timee + '\n'+ '\n')
    write.write(name + '\n')
    
    for i in results:
        
        results1 = results[i]
        write.write("Category = " + i + '\n')
        
        for j in results1:
            
            write.write(j + " = " + str(results1[j]) + '\n')
            
        write.write('\n')
        write.write('\n')
        
    write.close()

def balancedData(X,Y):
    
    cases = 0
    for i in range(len(Y)):
        if Y[i] == 1:
            cases += 1
            
    Xbalanced = np.zeros((2*cases,len(X.T)))
    Ybalanced = np.zeros(2*cases)
    controls = 0
    count = 0
    for i in range(len(Y)):
        if(Y[i] == 0 and controls < cases):
            Xbalanced[count,:] = X[i,:]
            Ybalanced[count] = Y[i]
            controls +=1
            count += 1
        elif Y[i] == 1:
            Xbalanced[count,:] = X[i,:]
            Ybalanced[count] = Y[i]
            count += 1
            
    print("len x = ", Xbalanced.shape)
    print("len y = ", Ybalanced.shape)
    
    return Xbalanced, Ybalanced
            

C:\Users\ANTONIS\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\ANTONIS\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\ANTONIS\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:

path = 'C:\\Users\\ANTONIS\\Desktop\\newSet\\maf\\maf = 0.05\\pvalue = 0.001\\'
#path = 'C:\\Users\\ANTONIS\\Desktop\\newSet\\maf\\maf = 0.05\\pvalue = 0.01\\'
#path = 'C:\\Users\\ANTONIS\\Desktop\\newSet\\maf\\maf = 0.05\\pvalue = 0.005\\'
#path = 'C:\\Users\\ANTONIS\\Desktop\\newSet\\maf\\maf = 0.05\\pvalue = 0.05\\'

numberOfChromosomes = 22#'ari8mos twn xromoswmatwn'
patientsTrain = {}
patientsTest = {}
allPatients = {}

chromosomes = {}

read = Read(path,numberOfChromosomes)
write = Write(path,numberOfChromosomes)

patients = read.readPatients('Patients.txt')
chromosomes = read.readSnps(".assoc")
write.writePatientsList(patients,'patient.txt')
write.writeSnpsList(chromosomes)


# run train_lgen bat and test_leg bat

In [3]:

snps = read.getListOfSnps()
ids = {} 


ids['patients'] = setIdToName(list(patients.keys()))
ids['snps'] = setIdToName(snps)


if os.path.exists(path + 'snpCode.txt'):
    print("mphka")
    X, Y = read.readSnpsCode(patients,ids)
    
else:
    
    write.writeSnpLog(read.getNumberOfPatients(),read.getNumberOfSnps(),chromosomes)
    X, Y = read.readSnpsCode(patients,ids)
    
X.shape

mphka
mphka2


(4980, 7799)

# Correlation

In [4]:
xTraining, xTest, yTraining1, yTest = train_test_split(X, Y, test_size=0.1, random_state=randint(0,2017))
print("mergex = ",X.shape)
print("xTrain = ",xTraining.shape)
print("xTest = ",xTest.shape)
print("yTrain = ",yTraining1.shape)
print("yTest = ",yTest.shape)

mergex =  (4980, 7799)
xTrain =  (4482, 7799)
xTest =  (498, 7799)
yTrain =  (4482,)
yTest =  (498,)


In [5]:
cor = Correlation(X)
#cor = RSquare(X)

#cor = Correlation(xTraining)
#cor = RSquare(xTraining)
snpReduc = {}

# Correlation high

In [6]:
snpReduc['high'] = cor.getHighCorrelationSnps(0.7,down=100,up=100)
write.writeSnpsUsed(snpReduc['high'],ids['snps']['idToName'],chromosomes,'high')

snpsIds =  0
idToName =  7799


# Correlation Low

In [6]:
down = 100
up = 100 
threshold = 0.7
snpReduc['low'] = cor.getLowCorrelationSnps(threshold, down=down,up=up)
write.writeSnpsUsed(snpReduc['low'],ids['snps']['idToName'],chromosomes,'low')

snpsIds =  883
idToName =  7799


In [8]:
'''print(ids['snps']['nameToId']['rs75570604'])
get = cor.getCorrMatrix()

summ = 0 

for i in range(len(get)):
    if get[6555,i] - 0.7 < 1e-10 :
        summ += 1'''

'''for i in range(len(get)):
    sum1 = 0
    for j in range(len(get.T)):
        if get[i,j] - 0.7 < 1e-10 :
            sum1 += 1
    if sum1 == len(get.T)-1:
        summ += 1
'''

'''for i in range(len(get)):
    sum1 = 0
    for j in range(len(get.T)):
        if get[i,j] - 0.7 > 1e-10 :
            sum1 += 1
    if sum1 > summ:
        summ = sum1'''


'''print(summ)
print(len(get))'''

'print(summ)\nprint(len(get))'

In [10]:
categories = []
#categories = ['low','high']
categories = ['low']
writeResults2={}
writeResults={}

In [10]:
'''featuresIdss = {}
featuresIdss['low'] = featuresIds(snpReduc['low'])'''


"featuresIdss = {}\nfeaturesIdss['low'] = featuresIds(snpReduc['low'])"

# SVM KERNEL

In [11]:
from sklearn.svm import NuSVC
for i in categories:
    
    r1 = {}
    r1['down'] = down
    r1['up'] = up
    r1['thres'] = threshold
    
    print("Category = ",i)
    
    XX = createNewTable(snpReduc[i],X)
    xTraining1 = createNewTable(snpReduc[i],xTraining)
    xTest1 = createNewTable(snpReduc[i],xTest)
    
    enc = OneHotEncoder(n_values =3) 
    enc.fit(XX) 
    XX = enc.fit_transform(XX)
    XX = XX.toarray()
    print(XX.shape)

    enc.fit(xTraining1) 
    xTraining1 = enc.fit_transform(xTraining1)
    xTraining1 = xTraining1.toarray()
    print(xTraining1.shape)

    enc.fit(xTest1) 
    xTest1 = enc.fit_transform(xTest1)
    xTest1 = xTest1.toarray()
    print(xTest1.shape)
    
    
    clf = SVC(kernel='linear')
    #clf = NuSVC(kernel='rbf',nu=0.01)
    clf.fit(xTraining1, yTraining1)
    yPredict2 = clf.predict(xTest1)
    print(metrics.accuracy_score(yTest,yPredict2))
    print(metrics.confusion_matrix(yTest,yPredict2))
    error2 = mean_squared_error(yTest, yPredict2)
    print("error 2 = ",error2)
    RMSE2 = mean_squared_error(yTest,yPredict2)**0.5
    print("RMSE2 = ",RMSE2)
    fpr, tpr, thresholds = metrics.roc_curve(yTest,yPredict2)
    print("AUC = ", metrics.auc(fpr,tpr))
    print("recal = ",metrics.recall_score(yTest,yPredict2))
    print("precision = ",metrics.precision_score(yTest,yPredict2))
    print("f1Score = ",f1_score(yTest, yPredict2, average='binary'))
    
    r1['confu'] = metrics.confusion_matrix(yTest,yPredict2)
    r1['accu'] = metrics.accuracy_score(yTest,yPredict2)
    r1['auc'] = metrics.auc(fpr,tpr)
    r1['recal'] = metrics.recall_score(yTest,yPredict2)
    r1['precision'] = metrics.precision_score(yTest,yPredict2)
    r1['f1'] = f1_score(yTest, yPredict2, average='binary')
    
    writeResults2[i] = r1
    print()

    results = crossValidiation(XX, Y, k = 10, classifier = clf, continious = False)
    results['down'] = down
    results['up'] = up
    results['thres'] = threshold
    print("accuracy = ",results['accuracy'])
    print("AUC = ", results['auc'])
    print("recal = ",results['recall'])
    print("precision = ",results['precision'])
    print("f1 = ",results['f1'])
    results['len snps'] = len(snpReduc[i])
    writeResults[i] = results
    
    print()
    
writeResult(path,'Linear',writeResults)
writeResultConf(path,'Linear',writeResults2)

Category =  low
new shape =  (4980, 6250)
new shape =  (4482, 6250)
new shape =  (498, 6250)
(4980, 18750)
(4482, 18750)
(498, 18750)
0.953815261044
[[388   6]
 [ 17  87]]
error 2 =  0.0461847389558
RMSE2 =  0.214906349268
AUC =  0.910655017571
recal =  0.836538461538
precision =  0.935483870968
f1Score =  0.883248730964

accuracy =  0.939959839357
AUC =  0.896714244094
recal =  0.823171610863
precision =  0.87548031077
f1 =  0.847611157466



In [12]:
#re = cross_val_score(clf, XX, Y, cv=10)
#print(sum(re)/10)

# LINEAR LOGISTIC REGRESSION 

In [11]:

for i in categories:
  
    r1 = {}
    
    print("Category = ",i)
    
    XX = createNewTable(snpReduc[i],X)
    
    xTraining1 = createNewTable(snpReduc[i],xTraining)
    xTest1 = createNewTable(snpReduc[i],xTest)
    
    ''' enc = OneHotEncoder(n_values =3) 
    enc.fit(XX) 
    XX = enc.fit_transform(XX)
    XX = XX.toarray()
    print(XX.shape)

    enc.fit(xTraining1) 
    xTraining1 = enc.fit_transform(xTraining1)
    xTraining1 = xTraining1.toarray()
    print(xTraining1.shape)

    enc.fit(xTest1) 
    xTest1 = enc.fit_transform(xTest1)
    xTest1 = xTest1.toarray()
    print(xTest1.shape)'''
    
    

    #lr_clf = linear_model.LogisticRegression()
    lr_clf = linear_model.LogisticRegressionCV()
    lr_clf.fit(xTraining1, yTraining1)
    yPredict4 = lr_clf.predict(xTest1)

    print(metrics.accuracy_score(yTest,yPredict4))
    print(metrics.confusion_matrix(yTest,yPredict4))
    error4 = mean_squared_error(yTest, yPredict4)
    print("error 4 = ",error4)
    RMSE4 = mean_squared_error(yTest,yPredict4)**0.5
    print("RMSE4 = ",RMSE4)

    fpr, tpr, thresholds = metrics.roc_curve(yTest,yPredict4)
    print("AUC = ", metrics.auc(fpr,tpr))
    print("recal = ",metrics.recall_score(yTest,yPredict4))
    print("precision = ",metrics.precision_score(yTest,yPredict4))
    print("f1Score = ",f1_score(yTest, yPredict4, average='binary'))
    
    r1['confu'] = metrics.confusion_matrix(yTest,yPredict4)
    r1['accu'] = metrics.accuracy_score(yTest,yPredict4)
    r1['auc'] = metrics.auc(fpr,tpr)
    r1['recal'] = metrics.recall_score(yTest,yPredict4)
    r1['precision'] = metrics.precision_score(yTest,yPredict4)
    r1['f1'] = f1_score(yTest, yPredict4, average='binary')
    writeResults2[i] = r1
    
    print()

    results = crossValidiation(XX, Y, k = 10, classifier = lr_clf, continious = False)
    results['down'] = down
    results['up'] = up
    results['thres'] = threshold
    print("accuracy = ",results['accuracy'])
    print("AUC = ", results['auc'])
    print("recal = ",results['recall'])
    print("precision = ",results['precision'])
    print("f1 = ",results['f1'])
    results['len snps'] = len(snpReduc[i])
    writeResults[i] = results
    
    print()
    
    
writeResult(path,'llrcv',writeResults)
writeResultConf(path,'llrcv',writeResults2)

Category =  low
new shape =  (4980, 883)
new shape =  (4482, 883)
new shape =  (498, 883)
0.955823293173
[[385   7]
 [ 15  91]]
error 4 =  0.0441767068273
RMSE4 =  0.210182555954
AUC =  0.92031671159
recal =  0.858490566038
precision =  0.928571428571
f1Score =  0.892156862745

accuracy =  0.936746987952
AUC =  0.880836368515
recal =  0.786179034288
precision =  0.893473966017
f1 =  0.835176061525



In [14]:
XX = createNewTable(snpReduc['low'],X)
lr_clf = linear_model.LogisticRegressionCV()
lr_clf.fit(XX, Y)

coefs=lr_clf.coef_[0]
top_20 = np.argpartition(coefs, -200)[-200:]
low_20 = np.argpartition(coefs, 200)[:200]
removedSnps = []

for i in range(len(snpReduc['low'])):
    if i not in top_20:
        removedSnps.append(snpReduc['low'][i])
        
snpReduc['lowTopReduced'] = removedSnps

removedSnps = []
for i in range(len(snpReduc['low'])):
    if i not in low_20:
        removedSnps.append(snpReduc['low'][i])

snpReduc['lowLowReduced'] = removedSnps


new shape =  (4980, 6250)


In [9]:
#XX = createNewTable(snpReduc['low'],X)
lr_clf = linear_model.LogisticRegressionCV()
lr_clf.fit(X, Y)

coefs=lr_clf.coef_[0]
top_30 = np.argpartition(coefs, -30)[-30:]
#low_20 = np.argpartition(coefs, 100)[:100]
removedSnps = []

#for i in range(len(snpReduc['low'])):
for i in range(len(ids['snps']['idToName'])):
    if i in top_30:
        #removedSnps.append(snpReduc['low'][i])
        removedSnps.append(i)

write.writeSnpsUsed(removedSnps,ids['snps']['idToName'],chromosomes,'top_30_all')


snpsIds =  30
idToName =  7799


In [16]:
from sklearn.naive_bayes import GaussianNB

for i in categories:
    
    r1 = {}
    
    print("Category = ",i)
    
    XX = createNewTable(snpReduc[i],X)
    xTraining1 = createNewTable(snpReduc[i],xTraining)
    xTest1 = createNewTable(snpReduc[i],xTest)
    
    enc = OneHotEncoder(n_values =3) 
    enc.fit(XX) 
    XX = enc.fit_transform(XX)
    XX = XX.toarray()
    print(XX.shape)

    enc.fit(xTraining1) 
    xTraining1 = enc.fit_transform(xTraining1)
    xTraining1 = xTraining1.toarray()
    print(xTraining1.shape)

    enc.fit(xTest1) 
    xTest1 = enc.fit_transform(xTest1)
    xTest1 = xTest1.toarray()
    print(xTest1.shape)
    

    gnb = GaussianNB()
    gnb.fit(xTraining1, yTraining1)
    yPredict6 = gnb.predict(xTest1)
    print(metrics.accuracy_score(yTest,yPredict6))
    print(metrics.confusion_matrix(yTest,yPredict6))
    error6 = mean_squared_error(yTest, yPredict6)
    print("error 6 = ",error6)
    RMSE6 = mean_squared_error(yTest,yPredict6)**0.5
    print("RMSE6 = ",RMSE6)

    fpr, tpr, thresholds = metrics.roc_curve(yTest,yPredict6)
    print("AUC = ", metrics.auc(fpr,tpr))
    print("recal = ",metrics.recall_score(yTest,yPredict6))
    print("precision = ",metrics.precision_score(yTest,yPredict6))
    print("f1Score = ",f1_score(yTest, yPredict6, average='binary'))
    
    r1['confu'] = metrics.confusion_matrix(yTest,yPredict6)
    r1['accu'] = metrics.accuracy_score(yTest,yPredict6)
    r1['auc'] = metrics.auc(fpr,tpr)
    r1['recal'] = metrics.recall_score(yTest,yPredict6)
    r1['precision'] = metrics.precision_score(yTest,yPredict6)
    r1['f1'] = f1_score(yTest, yPredict6, average='binary')
    
    writeResults2[i] = r1
    
    print()

    results = crossValidiation(XX, Y, k = 10,classifier = gnb, continious = False)
    results['down'] = down
    results['up'] = up
    results['thres'] = threshold
    print("accuracy = ",results['accuracy'])
    print("AUC = ", results['auc'])
    print("recal = ",results['recall'])
    print("precision = ",results['precision'])
    print("f1 = ",results['f1'])
    print()
    results['len snps'] = len(snpReduc[i])
    writeResults[i] = results
    
    print()
    
    
writeResult(path,'gnb',writeResults)
writeResultConf(path,'gnb',writeResults2)

Category =  low
new shape =  (4980, 6250)
new shape =  (4482, 6250)
new shape =  (498, 6250)
(4980, 18750)
(4482, 18750)
(498, 18750)
0.481927710843
[[146 248]
 [ 10  94]]
error 6 =  0.518072289157
RMSE6 =  0.719772387048
AUC =  0.63720226474
recal =  0.903846153846
precision =  0.27485380117
f1Score =  0.421524663677

accuracy =  0.449196787149
AUC =  0.615387642063
recal =  0.895662666008
precision =  0.257282501572
f1 =  0.399035847604




# BERNOULLI


In [17]:

'''xTraining1 = createNewTable(snpReduc['low'],xTraining)
lr_clf = linear_model.LogisticRegressionCV()
lr_clf.fit(xTraining1, yTraining1)

coefs=lr_clf.coef_[0]
top_20 = np.argpartition(coefs, -50)[-50:]
#top_20 = np.argpartition(coefs, 50)[:50]
removedSnps = []
for i in range(len(snpReduc['low'])):
    if i not in top_20:
        removedSnps.append(snpReduc['low'][i])'''


snpReduc['lowReduced'] = removedSnps

for i in categories:
   
    r1 = {}
    
    print("Category = ",i)
    print("snpReduc = ", len(snpReduc[i]))
   
   
  
    XX = createNewTable(snpReduc[i],X)
    xTraining1 = createNewTable(snpReduc[i],xTraining)
    xTest1 = createNewTable(snpReduc[i],xTest)

   
    enc = OneHotEncoder(n_values =3) 
    enc.fit(XX) 
    XX = enc.fit_transform(XX)
    XX = XX.toarray()
    print(XX.shape)

    enc.fit(xTraining1) 
    xTraining1 = enc.fit_transform(xTraining1)
    xTraining1 = xTraining1.toarray()
    print(xTraining1.shape)

    enc.fit(xTest1) 
    xTest1 = enc.fit_transform(xTest1)
    xTest1 = xTest1.toarray()
    print(xTest1.shape)
    

    bern = BernoulliNB()
    bern.fit(xTraining1, yTraining1)
    yPredict6 = bern.predict(xTest1)
    print(metrics.accuracy_score(yTest,yPredict6))
    print(metrics.confusion_matrix(yTest,yPredict6))
    error6 = mean_squared_error(yTest, yPredict6)
    print("error 6 = ",error6)
    RMSE6 = mean_squared_error(yTest,yPredict6)**0.5
    print("RMSE6 = ",RMSE6)

    fpr, tpr, thresholds = metrics.roc_curve(yTest,yPredict6)
    print("AUC = ", metrics.auc(fpr,tpr))
    print("recal = ",metrics.recall_score(yTest,yPredict6))
    print("precision = ",metrics.precision_score(yTest,yPredict6))
    print("f1Score = ",f1_score(yTest, yPredict6, average='binary'))
    
    r1['confu'] = metrics.confusion_matrix(yTest,yPredict6)
    r1['accu'] = metrics.accuracy_score(yTest,yPredict6)
    r1['auc'] = metrics.auc(fpr,tpr)
    r1['recal'] = metrics.recall_score(yTest,yPredict6)
    r1['precision'] = metrics.precision_score(yTest,yPredict6)
    r1['f1'] = f1_score(yTest, yPredict6, average='binary')
    
    writeResults2[i] = r1
    
    print()

    results = crossValidiation(XX, Y, k = 10,classifier = bern, continious = False)
    results['down'] = down
    results['up'] = up
    results['thres'] = threshold
    print("accuracy = ",results['accuracy'])
    print("AUC = ", results['auc'])
    print("recal = ",results['recall'])
    print("precision = ",results['precision'])
    print("f1 = ",results['f1'])
    print()
    results['len snps'] = len(snpReduc[i])
    writeResults[i] = results
    
    print()
    
    
writeResult(path,'bernoulli',writeResults)
writeResultConf(path,'bernoulli',writeResults2)

Category =  low
snpReduc =  6250
new shape =  (4980, 6250)
new shape =  (4482, 6250)
new shape =  (498, 6250)
(4980, 18750)
(4482, 18750)
(498, 18750)
0.847389558233
[[339  55]
 [ 21  83]]
error 6 =  0.152610441767
RMSE6 =  0.390653864395
AUC =  0.829241507224
recal =  0.798076923077
precision =  0.601449275362
f1Score =  0.685950413223

accuracy =  0.838554216867
AUC =  0.818829283419
recal =  0.785864851023
precision =  0.576997682545
f1 =  0.664856113196




In [18]:
'''l= [5,6,7,2,3,4,10,6,7,9,8,5]
top_3 = np.argpartition(l,3)[:3]
print(top_3)
removedSnps=[]
for i in range(len(l)):
    if i not in top_3:
        removedSnps.append(l[i])
print(removedSnps)'''

'l= [5,6,7,2,3,4,10,6,7,9,8,5]\ntop_3 = np.argpartition(l,3)[:3]\nprint(top_3)\nremovedSnps=[]\nfor i in range(len(l)):\n    if i not in top_3:\n        removedSnps.append(l[i])\nprint(removedSnps)'

In [19]:
XX = createNewTable(snpReduc['low'],X)
#enc = OneHotEncoder(n_values =3) 
#enc.fit(XX) 
#XX = enc.fit_transform(XX)
#XX = XX.toarray()
print(XX.shape)

re = cross_val_score(bern, XX, Y, cv=10)
print(sum(re)/10)

print()
XX = X
#enc = OneHotEncoder(n_values =3) 
#enc.fit(XX) 
#XX = enc.fit_transform(XX)
#XX = XX.toarray()
print(XX.shape)
re = cross_val_score(bern, XX, Y, cv=10)
print(sum(re)/10)


new shape =  (4980, 6250)
(4980, 6250)
0.832732670176

(4980, 7799)
0.750796361334


# TREE

In [20]:
from sklearn import tree

for i in categories:
    
    r1 = {}
    
    print("Category = ",i)
    
    XX = createNewTable(snpReduc[i],X)
    xTraining1 = createNewTable(snpReduc[i],xTraining)
    xTest1 = createNewTable(snpReduc[i],xTest)
    
    enc = OneHotEncoder(n_values =3) 
    enc.fit(XX) 
    XX = enc.fit_transform(XX)
    XX = XX.toarray()
    print(XX.shape)

    enc.fit(xTraining1) 
    xTraining1 = enc.fit_transform(xTraining1)
    xTraining1 = xTraining1.toarray()
    print(xTraining1.shape)

    enc.fit(xTest1) 
    xTest1 = enc.fit_transform(xTest1)
    xTest1 = xTest1.toarray()
    print(xTest1.shape)
    
    

    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(xTraining1, yTraining1)
    yPredict4 = clf.predict(xTest1)

    print(metrics.accuracy_score(yTest,yPredict4))
    print(metrics.confusion_matrix(yTest,yPredict4))
    error4 = mean_squared_error(yTest, yPredict4)
    print("error 4 = ",error4)
    RMSE4 = mean_squared_error(yTest,yPredict4)**0.5
    print("RMSE4 = ",RMSE4)

    fpr, tpr, thresholds = metrics.roc_curve(yTest,yPredict4)
    print("AUC = ", metrics.auc(fpr,tpr))
    print("recal = ",metrics.recall_score(yTest,yPredict4))
    print("precision = ",metrics.precision_score(yTest,yPredict4))
    print("f1Score = ",f1_score(yTest, yPredict4, average='binary'))
    
    r1['confu'] = metrics.confusion_matrix(yTest,yPredict4)
    r1['accu'] = metrics.accuracy_score(yTest,yPredict4)
    r1['auc'] = metrics.auc(fpr,tpr)
    r1['recal'] = metrics.recall_score(yTest,yPredict4)
    r1['precision'] = metrics.precision_score(yTest,yPredict4)
    r1['f1'] = f1_score(yTest, yPredict4, average='binary')
    writeResults2[i] = r1
    
    print()

    results = crossValidiation(XX, Y, k = 10, classifier = clf, continious = False)
    results['down'] = down
    results['up'] = up
    results['thres'] = threshold
    print("accuracy = ",results['accuracy'])
    print("AUC = ", results['auc'])
    print("recal = ",results['recall'])
    print("precision = ",results['precision'])
    print("f1 = ",results['f1'])
    results['len snps'] = len(snpReduc[i])
    writeResults[i] = results
    
    print()
    
    
writeResult(path,'tree',writeResults)
writeResultConf(path,'tree',writeResults2)

Category =  low
new shape =  (4980, 6250)
new shape =  (4482, 6250)
new shape =  (498, 6250)
(4980, 18750)
(4482, 18750)
(498, 18750)
0.664658634538
[[307  87]
 [ 80  24]]
error 4 =  0.335341365462
RMSE4 =  0.579086664897
AUC =  0.504978524014
recal =  0.230769230769
precision =  0.216216216216
f1Score =  0.223255813953

accuracy =  0.691164658635
AUC =  0.53240016438
recal =  0.263084388208
precision =  0.255485734475
f1 =  0.257558551071



In [21]:
'''import graphviz
xTraining1 = createNewTable(snpReduc['low'],xTraining)
clf = clf.fit(xTraining1, yTraining1)
treeee = open(path+'treeLayers.dot','w')
dot_data = tree.export_graphviz(clf, out_file=treeee, 
                         feature_names=snpReduc['low'],  
                         class_names=yTraining1,  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = graphviz.Source(dot_data)  
'''

"import graphviz\nxTraining1 = createNewTable(snpReduc['low'],xTraining)\nclf = clf.fit(xTraining1, yTraining1)\ntreeee = open(path+'treeLayers.dot','w')\ndot_data = tree.export_graphviz(clf, out_file=treeee, \n                         feature_names=snpReduc['low'],  \n                         class_names=yTraining1,  \n                         filled=True, rounded=True,  \n                         special_characters=True)  \ngraph = graphviz.Source(dot_data)  \n"

# ALL


In [22]:

r1 = {}
writeResults2={}
writeResults={}

enc = OneHotEncoder(n_values =3) 
enc.fit(X) 
XX = enc.fit_transform(X)
XX = XX.toarray()
print(XX.shape)

enc.fit(xTraining) 
xTraining1 = enc.fit_transform(xTraining)
xTraining1 = xTraining1.toarray()
print(xTraining1.shape)

enc.fit(xTest) 
xTest1 = enc.fit_transform(xTest)
xTest1 = xTest1.toarray()
print(xTest1.shape)
    

bern = BernoulliNB()
bern.fit(xTraining1, yTraining1)
yPredict6 = bern.predict(xTest1)
print(metrics.accuracy_score(yTest,yPredict6))
print(metrics.confusion_matrix(yTest,yPredict6))
error6 = mean_squared_error(yTest, yPredict6)
print("error 6 = ",error6)
RMSE6 = mean_squared_error(yTest,yPredict6)**0.5
print("RMSE6 = ",RMSE6)

fpr, tpr, thresholds = metrics.roc_curve(yTest,yPredict6)
print("AUC = ", metrics.auc(fpr,tpr))
print("recal = ",metrics.recall_score(yTest,yPredict6))
print("precision = ",metrics.precision_score(yTest,yPredict6))
print("f1Score = ",f1_score(yTest, yPredict6, average='binary'))

r1['confu'] = metrics.confusion_matrix(yTest,yPredict6)
r1['accu'] = metrics.accuracy_score(yTest,yPredict6)
r1['auc'] = metrics.auc(fpr,tpr)
r1['recal'] = metrics.recall_score(yTest,yPredict6)
r1['precision'] = metrics.precision_score(yTest,yPredict6)
r1['f1'] = f1_score(yTest, yPredict6, average='binary')


writeResults2['all'] = r1
    
print()

results = crossValidiation(XX, Y, k = 10,classifier = bern, continious = False)
results['down'] = down
results['up'] = up
results['thres'] = threshold    
print("accuracy = ",results['accuracy'])
print("AUC = ", results['auc'])
print("recal = ",results['recall'])
print("precision = ",results['precision'])
print("f1 = ",results['f1'])


results['len snps'] = len(XX.T)
writeResults['all'] = results
    
print()
    
    
writeResult(path,'bernoulli_all',writeResults)
writeResultConf(path,'bernoulli_all',writeResults2)


(4980, 23397)
(4482, 23397)
(498, 23397)
0.787148594378
[[318  76]
 [ 30  74]]
error 6 =  0.212851405622
RMSE6 =  0.461358218332
AUC =  0.759322530262
recal =  0.711538461538
precision =  0.493333333333
f1Score =  0.582677165354

accuracy =  0.758835341365
AUC =  0.738527649674
recal =  0.704272079045
precision =  0.442464800218
f1 =  0.54294527503



In [23]:
r1 = {}
writeResults2={}
writeResults={}

enc = OneHotEncoder(n_values =3) 
enc.fit(X) 
XX = enc.fit_transform(X)
XX = XX.toarray()
print(XX.shape)

enc.fit(xTraining) 
xTraining1 = enc.fit_transform(xTraining)
xTraining1 = xTraining1.toarray()
print(xTraining1.shape)

enc.fit(xTest) 
xTest1 = enc.fit_transform(xTest)
xTest1 = xTest1.toarray()
print(xTest1.shape)
    

bern = SVC(kernel='linear')
bern.fit(xTraining1, yTraining1)
yPredict6 = bern.predict(xTest1)
print(metrics.accuracy_score(yTest,yPredict6))
print(metrics.confusion_matrix(yTest,yPredict6))
error6 = mean_squared_error(yTest, yPredict6)
print("error 6 = ",error6)
RMSE6 = mean_squared_error(yTest,yPredict6)**0.5
print("RMSE6 = ",RMSE6)

fpr, tpr, thresholds = metrics.roc_curve(yTest,yPredict6)
print("AUC = ", metrics.auc(fpr,tpr))
print("recal = ",metrics.recall_score(yTest,yPredict6))
print("precision = ",metrics.precision_score(yTest,yPredict6))
print("f1Score = ",f1_score(yTest, yPredict6, average='binary'))

r1['confu'] = metrics.confusion_matrix(yTest,yPredict6)
r1['accu'] = metrics.accuracy_score(yTest,yPredict6)
r1['auc'] = metrics.auc(fpr,tpr)
r1['recal'] = metrics.recall_score(yTest,yPredict6)
r1['precision'] = metrics.precision_score(yTest,yPredict6)
r1['f1'] = f1_score(yTest, yPredict6, average='binary')
   
writeResults2['all'] = r1
    
print()

results = crossValidiation(XX, Y, k = 10,classifier = bern, continious = False)
results['down'] = down
results['up'] = up
results['thres'] = threshold  
print("accuracy = ",results['accuracy'])
print("AUC = ", results['auc'])
print("recal = ",results['recall'])
print("precision = ",results['precision'])
print("f1 = ",results['f1'])


results['len snps'] = len(XX.T)
writeResults['all'] = results
    
print()
    
    
writeResult(path,'svm_all',writeResults)
writeResultConf(path,'svm_all',writeResults2)

(4980, 23397)
(4482, 23397)
(498, 23397)
0.95983935743
[[388   6]
 [ 14  90]]
error 6 =  0.0401606425703
RMSE6 =  0.200401204014
AUC =  0.925078094494
recal =  0.865384615385
precision =  0.9375
f1Score =  0.9

accuracy =  0.942771084337
AUC =  0.898998372861
recal =  0.824938596318
precision =  0.886024460004
f1 =  0.853885877663



In [12]:
r1 = {}
writeResults2={}
writeResults={}

enc = OneHotEncoder(n_values =3) 
enc.fit(X) 
XX = enc.fit_transform(X)
XX = XX.toarray()
print(XX.shape)

enc.fit(xTraining) 
xTraining1 = enc.fit_transform(xTraining)
xTraining1 = xTraining1.toarray()
print(xTraining1.shape)

enc.fit(xTest) 
xTest1 = enc.fit_transform(xTest)
xTest1 = xTest1.toarray()
print(xTest1.shape)
    

bern = linear_model.LogisticRegressionCV()
bern.fit(xTraining1, yTraining1)
yPredict6 = bern.predict(xTest1)
print(metrics.accuracy_score(yTest,yPredict6))
print(metrics.confusion_matrix(yTest,yPredict6))
error6 = mean_squared_error(yTest, yPredict6)
print("error 6 = ",error6)
RMSE6 = mean_squared_error(yTest,yPredict6)**0.5
print("RMSE6 = ",RMSE6)

fpr, tpr, thresholds = metrics.roc_curve(yTest,yPredict6)
print("AUC = ", metrics.auc(fpr,tpr))
print("recal = ",metrics.recall_score(yTest,yPredict6))
print("precision = ",metrics.precision_score(yTest,yPredict6))
print("f1Score = ",f1_score(yTest, yPredict6, average='binary'))

r1['confu'] = metrics.confusion_matrix(yTest,yPredict6)
r1['accu'] = metrics.accuracy_score(yTest,yPredict6)
r1['auc'] = metrics.auc(fpr,tpr)
r1['recal'] = metrics.recall_score(yTest,yPredict6)
r1['precision'] = metrics.precision_score(yTest,yPredict6)
r1['f1'] = f1_score(yTest, yPredict6, average='binary')
   
#writeResults2['all'] = r1

print()

results = crossValidiation(XX, Y, k = 10,classifier = bern, continious = False)
results['down'] = down
results['up'] = up
results['thres'] = threshold    
print("accuracy = ",results['accuracy'])
print("AUC = ", results['auc'])
print("recal = ",results['recall'])
print("precision = ",results['precision'])
print("f1 = ",results['f1'])


results['len snps'] = len(XX.T)
writeResults['all'] = results
    
print()
    
    
writeResult(path,'llr_all_CV',writeResults)
writeResultConf(path,'llr_all_CV',writeResults2)

(4980, 23397)
(4482, 23397)
(498, 23397)
0.943775100402
[[379  13]
 [ 15  91]]
error 6 =  0.0562248995984
RMSE6 =  0.237117902315
AUC =  0.912663650366
recal =  0.858490566038
precision =  0.875
f1Score =  0.866666666667

accuracy =  0.939357429719
AUC =  0.891733397204
recal =  0.811332233043
precision =  0.883734192557
f1 =  0.845476890778



In [25]:
r1 = {}
writeResults2={}
writeResults={}

#enc = OneHotEncoder(n_values =3) 
#enc.fit(X) 
#XX = enc.fit_transform(X)
#XX = XX.toarray()
#print(XX.shape)

#enc.fit(xTraining) 
#xTraining1 = enc.fit_transform(xTraining)
#xTraining1 = xTraining1.toarray()
#print(xTraining1.shape)

#enc.fit(xTest) 
#xTest1 = enc.fit_transform(xTest)
#xTest1 = xTest1.toarray()
#print(xTest1.shape)
    

bern = tree.DecisionTreeClassifier()
bern.fit(xTraining1, yTraining1)
yPredict6 = bern.predict(xTest1)
print(metrics.accuracy_score(yTest,yPredict6))
print(metrics.confusion_matrix(yTest,yPredict6))
error6 = mean_squared_error(yTest, yPredict6)
print("error 6 = ",error6)
RMSE6 = mean_squared_error(yTest,yPredict6)**0.5
print("RMSE6 = ",RMSE6)

fpr, tpr, thresholds = metrics.roc_curve(yTest,yPredict6)
print("AUC = ", metrics.auc(fpr,tpr))
print("recal = ",metrics.recall_score(yTest,yPredict6))
print("precision = ",metrics.precision_score(yTest,yPredict6))
print("f1Score = ",f1_score(yTest, yPredict6, average='binary'))

r1['confu'] = metrics.confusion_matrix(yTest,yPredict6)
r1['accu'] = metrics.accuracy_score(yTest,yPredict6)
r1['auc'] = metrics.auc(fpr,tpr)
r1['recal'] = metrics.recall_score(yTest,yPredict6)
r1['precision'] = metrics.precision_score(yTest,yPredict6)
r1['f1'] = f1_score(yTest, yPredict6, average='binary')
   
writeResults2['all'] = r1

print()

results = crossValidiation(XX, Y, k = 10,classifier = bern, continious = False)
results['down'] = down
results['up'] = up
results['thres'] = threshold    
print("accuracy = ",results['accuracy'])
print("AUC = ", results['auc'])
print("recal = ",results['recall'])
print("precision = ",results['precision'])
print("f1 = ",results['f1'])


results['len snps'] = len(XX.T)
writeResults['all'] = results
    
print()
    
    
writeResult(path,'tree_all',writeResults)
writeResultConf(path,'tree_all',writeResults2)

0.682730923695
[[316  78]
 [ 80  24]]
error 6 =  0.317269076305
RMSE6 =  0.563266434563
AUC =  0.516399843811
recal =  0.230769230769
precision =  0.235294117647
f1Score =  0.233009708738

accuracy =  0.691365461847
AUC =  0.532243804504
recal =  0.2637398597
precision =  0.254403022427
f1 =  0.258130699244

